In [1]:
import os
import torch
import json
import random

from datasets import Dataset
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
torch.cuda.empty_cache() 


In [4]:
def load_dataset(file_path):
    with open(file_path, 'r') as f:
        dataset = json.load(f)
    return dataset

# Example usage:
dataset_file = "output_llama2.json"
my_dataset=load_dataset(dataset_file)

# Now my_dataset is a dictionary where each entry corresponds to a sample
# print(my_dataset[0])

hf_dataset = Dataset.from_dict({'text': [example['text'] for example in my_dataset]})
print(hf_dataset[0])

{'text': '<s>[INST]<<SYS>> Given characteristics about a patient, where applicable, provide the APR Severity of Illness Description: <</SYS>> Provide the APR Severity of Illness Description based on values provided: Health Service Area:Capital/Adiron, Hospital County:Clinton, Operating Certificate Number:901001.0, Facility ID:135.0, Facility Name:The University of Vermont Health Network - Champlain Valley Physicians, Age Group:70 or Older, Zip Code - 3 digits:129, Gender:M, Race:White, Ethnicity:Spanish/Hispanic, Length of Stay:17, Type of Admission:Emergency, Patient Disposition:Expired, Discharge Year:2013, CCS Diagnosis Code:122, CCS Diagnosis Description:Pneumonia (except that caused by tuberculosis or sexually transmitted disease), CCS Procedure Code:0, CCS Procedure Description:NO PROC, APR DRG Code:137, APR DRG Description:Major respiratory infections & inflammations, APR MDC Description:Diseases and Disorders of the Respiratory System, APR Risk of Mortality:Major, APR Medical S

**Extracting 30 Random samples for experimentation**

In [5]:
#overring the hf_dataset to 30 new samples
random_samples = random.sample(my_dataset, 30)
hf_dataset=random_samples
#this was the fix
hf_dataset = Dataset.from_dict({'text': [example['text'] for example in random_samples]})
len(hf_dataset)


30

In [6]:
hf_dataset

Dataset({
    features: ['text'],
    num_rows: 30
})

In [7]:
#saving 30 ranndom as json output
if isinstance(my_dataset[0], dict):
    # Elements are already dictionaries, no conversion needed.
    data_to_save = random_samples
else:
    # Elements are not dictionaries, convert to a list of dictionaries with an "item" key.
    data_to_save = [{'item': sample} for sample in random_samples]

# Save data to JSON file
with open('30_random_samples.json', 'w') as outfile:
    json.dump(data_to_save, outfile)

print("Random samples saved to 30_random_samples.json")

Random samples saved to 30_random_samples.json


In [8]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"


# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False



# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3


# Optimizer to use
# optim = "paged_adamw_32bit"
optim = "rmsprop_bnb"
# rmsprop_bnb
# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

## Experiment 1: lr=0.001 Batch = 3

In [9]:
#Creating an Eval dataset and rerunning
#selecting 10 random samples for evaluation
random_samples = random.sample(my_dataset, 10)
eval_dataset=random_samples
#this was the fix
eval_dataset = Dataset.from_dict({'text': [example['text'] for example in random_samples]})
len(eval_dataset)

10

In [10]:
# Load dataset (you can process it here)
# dataset = load_dataset(dataset_name, split="train")
from transformers.trainer_callback import EarlyStoppingCallback
#Setting New Parameters
per_device_train_batch_size = 1

per_device_eval_batch_size = 1

learning_rate = 0.001

weight_decay = 0.001

num_train_epochs = 1

# Log every X updates steps
logging_steps = 3

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Training arguments with early stopping criteria
training_arguments = TrainingArguments(
    load_best_model_at_end = True,
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,  # Maintain for limiting total training steps
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    # Early stopping parameters
    eval_steps=logging_steps * 2,  # Evaluate validation set every few training steps
    evaluation_strategy="steps",
    # Perform evaluation after each eval_steps
)

# SFTTrainer with early stopping logic
# patience=2  # Stop training if validation loss doesn't improve for 2 evaluations
# class EarlyStoppingCallback():
#     def __init__(self, patience=2):
#         self.patience = patience
#         self.best_score = None
#         self.counter = 0

#     def on_evaluate(self, eval_result, **kwargs):
#         # Track validation loss
#         current_score = eval_result.get("eval_loss")
#         if self.best_score is None or current_score < self.best_score:
#             self.best_score = current_score
#             self.counter = 0  # Reset counter on improvement
#         else:
#             self.counter += 1
#             if self.counter >= self.patience:
#                 print(f"Early stopping triggered after {self.patience} evaluations without improvement.")
#                 self.trainer.control = TrainingControl.STOP_EARLY

# Create EarlyStopping callback instance
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=patience)
# Train model with early stopping callback
trainer = SFTTrainer(
    model=model,
    train_dataset=hf_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    
)

# Train model
# trainer.train()

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/programmer/Documents/orcawise/facebook_llama2/llama_env/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/programmer/Documents/orcawise/facebook_llama2/llama_env/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be 

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/home/programmer/Documents/orcawise/facebook_llama2/llama_env/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
patience = 1
best_eval_loss = float('inf')

In [12]:
for epoch in range(3):
    print("epoch number: 1", epoch)
    # Train for one epoch
    trainer.train()
    eval_results = trainer.evaluate()
    eval_loss = eval_results['eval_loss']
    if eval_loss < best_eval_loss:
        best_eval_loss = eval_loss
        no_improvement_counter = 0
        # Save the model if it's the best so far
        trainer.save_model("best_model")
    else:
        no_improvement_counter += 1
    
    if no_improvement_counter >= patience:
        print(f"No improvement for {patience} epochs. Stopping training.")
        break

print("Training is complete")

epoch number: 1


Step,Training Loss,Validation Loss
6,3.198500,1.577319
12,1.226100,0.962921
18,0.493900,0.475221
24,1.559300,1.257631
30,1.061400,1.082540


epoch number: 1


Step,Training Loss,Validation Loss
6,0.474600,0.497114
12,0.320100,0.451211
18,0.292100,0.313466
24,0.314600,0.297279
30,0.248900,0.293450


epoch number: 1


Step,Training Loss,Validation Loss
6,0.292700,0.400868
12,0.256300,0.347605
18,0.272900,0.302644
24,0.279300,0.281562
30,0.202000,0.280440


Training is complete
